In [1]:
# -*- coding: utf-8 -*-
"""
Cleans AVL data by removing any pings that report a "next stop" that is not valid for the trip referenced.
"""

import os
import pandas as pd 
import sys
sys.path.append('/gpfs2/projects/project-bus_capstone_2016/workspace/mu529/bus-Capstone')

# these two modules are homemade
import gtfs
os.chdir('/gpfs2/projects/project-bus_capstone_2016/workspace/share')

In [2]:
# get all the schedule data. (subset can be created later)
trips = gtfs.load_trips('2015-12-03','gtfs/')
# stops = gtfs.load_stops('2015-12-03','gtfs/')
stop_times, tz_sched = gtfs.load_stop_times('2015-12-03','gtfs/')
tcal=gtfs.TransitCalendar('2015-12-03')
thurs_services = tcal.get_service_ids('2015-12-03')
thurs_trips = trips.service_id.isin(thurs_services)
thurs_stops = stop_times.reset_index().set_index('trip_id').loc[thurs_trips]
print 'Finished loading GTFS data.'

Finished loading GTFS data.


In [3]:
valid_stops = thurs_stops.groupby(level=0)['stop_id'].apply(list)

In [4]:
# get the sample of parsed AVL data.  Beware, large files take more time.
bustime = pd.read_csv('spark_parse/1203.txt',header=None)

# beware!  Bonan is still working on organizing the extract files.  these columns may change.
bustime.columns = ['route','lat','lon','timestamp','vehicle_id','trip_id','trip_date','shape_id',
                   'next_stop_id','dist_from_stop','stop_dist_on_trip','presentable_dist']

bustime.drop_duplicates(['vehicle_id','timestamp'],inplace=True)

In [5]:
bustime.shape

(1645578, 12)

In [6]:
bustime['trip_id'] = bustime['trip_id'].str.replace('MTA NYCT_','')
bustime['trip_id'] = bustime['trip_id'].str.replace('MTABC_','')
bustime['next_stop_id'] = bustime['next_stop_id'].str.replace('MTA_','')

In [7]:
cleaned = bustime.merge(pd.DataFrame(valid_stops),left_on='trip_id',right_index=True,how='left')

In [10]:
cleaned.shape

(1645578, 13)

In [11]:
def valid_stop(row):
    try:
        return int(row.next_stop_id) in row.stop_id
    except:
        return False

In [12]:
masker = cleaned.apply(valid_stop,axis=1)

In [17]:
cleaned.drop('stop_id',axis=1,inplace=True)

In [18]:
cleaned = cleaned[masker]

In [19]:
cleaned.to_csv('spark_parse/1203_cleaned.csv')